## Read Input and prepare data

In [80]:
ventLines = open("ventLines.txt") do file
    data = Int64[]
    nlines =0 #line counter
    coordinates=Array{Int}(undef,4)
    for line in eachline(file)
        nlines+=1
        splitString = split(line)
        coordinates[1:2]=parse.(Int,split(splitString[1],','))
        coordinates[3:4]=parse.(Int,split(splitString[3],','))
        append!(data,coordinates)
    end
    (reshape(data,4,nlines))
end;

In [81]:
ventLines

4×500 Array{Int64,2}:
 599  435  768  845  167   45  780  310  …   47  549  944  713   83  814  980
 531  904  714  552  680  887  295  539     103  880   15  456  865  585  439
 599  435  768  596  167   45  179  602     897  942   44  713  239  814  685
  32  489  187  801  445  346  896  831     953  487  915  402  865  105  439

In [181]:
ventLinesTest = open("ventLinesTest.txt") do file
    data = Int64[]
    nlines =0 #line counter
    coordinates=Array{Int}(undef,4)
    for line in eachline(file)
        nlines+=1
        splitString = split(line)
        coordinates[1:2]=parse.(Int,split(splitString[1],','))
        coordinates[3:4]=parse.(Int,split(splitString[3],','))
        append!(data,coordinates)
    end
    (reshape(data,4,nlines))
end;

In [182]:
ventLinesTest

4×10 Array{Int64,2}:
 0  8  9  2  7  6  0  3  0  5
 9  0  4  2  0  4  9  4  0  5
 5  0  3  2  7  2  2  1  8  8
 9  8  4  1  4  0  9  4  8  2

## Part 1

Find the maximum grid size in each direction

In [172]:
println(maximum(ventLines[1,:]))
println(maximum(ventLines[2,:]))
println(maximum(ventLines[3,:]))
println(maximum(ventLines[4,:]))

988
988
989
989


In [178]:
println(minimum(ventLines[1,:]))
println(minimum(ventLines[2,:]))
println(minimum(ventLines[3,:]))
println(minimum(ventLines[4,:]))

11
14
10
10


In [173]:
xMax = max(maximum(ventLines[1,:]),maximum(ventLines[3,:]))
yMax = max(maximum(ventLines[2,:]),maximum(ventLines[4,:]))
xMax,yMax

(989, 989)

In [202]:
function mapOceanFloor(data)
    xMax = max(maximum(data[1,:]),maximum(data[3,:]))
    yMax = max(maximum(data[2,:]),maximum(data[4,:]))
    map=zeros(Int,xMax,yMax)
    dataSize = size(data)
    
    #loop over all vent lines, act only on lines with x1==x2 or y1==y2
    for i in 1:dataSize[2]
        if data[1,i] == data[3,i]
            #check if coordinates are in ascending or descending order and 
            if data[2,i]>data[4,i]
                map[data[1,i],data[4,i]:data[2,i]] .+= 1
            else
                map[data[1,i],data[2,i]:data[4,i]] .+= 1
            end
        elseif data[2,i] == data[4,i]
            if data[1,i]>data[3,i]
                map[data[3,i]:data[1,i],data[2,i]] .+= 1
            else
                map[data[1,i]:data[3,i],data[2,i]] .+= 1
            end
        end
    end
    
    return map
end

mapOceanFloor (generic function with 1 method)

In [203]:
oceanFloor = mapOceanFloor(ventLines.+1);

In [215]:
count(x->x>1,oceanFloor)

7644

In [213]:
oceanFloorTest = mapOceanFloor(ventLinesTest.+1);

In [214]:
transpose(oceanFloorTest)

10×10 LinearAlgebra.Transpose{Int64,Array{Int64,2}}:
 0  0  0  0  0  0  0  1  0  0
 0  0  1  0  0  0  0  1  0  0
 0  0  1  0  0  0  0  1  0  0
 0  0  0  0  0  0  0  1  0  0
 0  1  1  2  1  1  1  2  1  1
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0
 2  2  2  1  1  1  0  0  0  0

In [210]:
count(x->x>1,oceanFloorTest)

5

#### Some trials on broadcasting addition operations

In [234]:
map=zeros(Int,5,5)

5×5 Array{Int64,2}:
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0
 0  0  0  0  0

In [235]:
map[3,4:-1:2].+=1 #ones(Int,3)
map[2:4,3].+=1 #ones(Int,3)

3-element view(::Array{Int64,2}, 2:4, 3) with eltype Int64:
 1
 2
 1

In [236]:
map[1,1]+=1

1

In [237]:
map

5×5 Array{Int64,2}:
 1  0  0  0  0
 0  0  1  0  0
 0  1  2  1  0
 0  0  1  0  0
 0  0  0  0  0

## Part 2

In [254]:
function mapOceanFloorDiag(data)
    xMax = max(maximum(data[1,:]),maximum(data[3,:]))
    yMax = max(maximum(data[2,:]),maximum(data[4,:]))
    map=zeros(Int,xMax,yMax)
    dataSize = size(data)
    
    #loop over all vent lines 
    for i in 1:dataSize[2]
        #horizontal lines
        if data[1,i] == data[3,i] 
            #check if coordinates are in ascending or descending order and 
            if data[2,i]>data[4,i]
                map[data[1,i],data[4,i]:data[2,i]] .+= 1
            else
                map[data[1,i],data[2,i]:data[4,i]] .+= 1
            end
        #vertical lines
        elseif data[2,i] == data[4,i] 
            if data[1,i]>data[3,i]
                map[data[3,i]:data[1,i],data[2,i]] .+= 1
            else
                map[data[1,i]:data[3,i],data[2,i]] .+= 1
            end
        #diagonal lines
        else
            #determine if either coordinate direction is in descending order
            incx = 1
            incy = 1
            if data[1,i]>data[3,i]
                incx = -1
            end
            if data[2,i]>data[4,i]
                incy = -1
            end 
            dx = abs(data[1,i] - data[3,i])
            for j in 0:dx     
                    map[data[1,i]+incx*j,data[2,i]+incy*j] +=1
            end
                    
        end
    end
    
    return map
end

mapOceanFloorDiag (generic function with 1 method)

In [251]:
oceanFloorTest = mapOceanFloorDiag(ventLinesTest.+1);

line 1 is vertical
line 2 is diagonal
line 3 is vertical
line 4 is horizontal
line 5 is horizontal
line 6 is diagonal
line 7 is vertical
line 8 is vertical
line 9 is diagonal
line 10 is diagonal


In [252]:
transpose(oceanFloorTest)

10×10 LinearAlgebra.Transpose{Int64,Array{Int64,2}}:
 1  0  1  0  0  0  0  1  1  0
 0  1  1  1  0  0  0  2  0  0
 0  0  2  0  1  0  1  1  1  0
 0  0  0  1  0  2  0  2  0  0
 0  1  1  2  3  1  3  2  1  1
 0  0  0  1  0  2  0  0  0  0
 0  0  1  0  0  0  1  0  0  0
 0  1  0  0  0  0  0  1  0  0
 1  0  0  0  0  0  0  0  1  0
 2  2  2  1  1  1  0  0  0  0

In [253]:
count(x->x>1,oceanFloorTest)

12

In [257]:
oceanFloor = mapOceanFloorDiag(ventLines.+1);

In [258]:
count(x->x>1,oceanFloor)

18627